In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import typing as t
from fdua_competition.vectorstore import build_vectorstore
from fdua_competition.enums import Mode, VectorStoreOption
from fdua_competition.rag import ResearchAssistant, RAG
from fdua_competition.enums import ChatModelOption
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.vectorstores.base import VectorStore

In [3]:
vectorstore = build_vectorstore("one_assistant_test", Mode.TEST, VectorStoreOption.CHROMA)

[prepare_vectorstore] chroma: /fdua-competition/.fdua-competition/vectorstores/chroma
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/9.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/8.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/10.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/6.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/7.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/5.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/4.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-co

In [36]:
# RAGクラスを継承して, 新しいRAGを作成する例

## レスポンスの型式を定義
class FactCheckerResponse(BaseModel):
    score: str = Field(description="スコア")
    reason: str = Field(description="理由")
    reference: list[str] = Field(description="[参照した文章]")


# RAGクラスを継承して新しいRAGクラスを定義
# prompt_template(), build_payload(), output_structure() メソッドをオーバーライドする
class FactChecker(RAG):
    def __init__(
        self, vectorstore: VectorStore, chat_model_option: ChatModelOption = ChatModelOption.AZURE, language: str = "japanese"
    ):
        super().__init__(vectorstore=vectorstore, chat_model_option=chat_model_option)
        self.language = language

    @property
    def prompt_template(self) -> ChatPromptTemplate:
        """プロンプトのテンプレート"""
        return ChatPromptTemplate.from_messages(
            [
                ("system", "情報の正確さを与えられたコンテキストのみ使ってを10点満点で評価して. "),
                ("system", "コンテキスト:\n{context}"),
                ("user", "チェック対象:\n{llm_response}"),
            ]
        )

    def build_payload(self, query: str) -> dict[str, t.Any]:
        """プロンプトテンプレートをこの辞書で埋める"""
        return {"llm_response": query, "language": self.language}

    @property
    def output_structure(self) -> BaseModel:
        """出力型式を定義する"""
        return FactCheckerResponse


# [end: research_assistant]

In [37]:
assistant = ResearchAssistant(vectorstore)

In [38]:
res = assistant.invoke("電通グループの創業は何年ですか？")
print(res)

query='電通グループの創業は何年ですか？' response='電通グループは1901年に創業されました。' reason='創業の年は文中で日本広告社として1901年に創設されたと記述されています。' organization_name='電通グループ' contexts=['年に創設された日本広告1901光永星郎の手によって・創業者\u3000']


In [39]:
fact_checker = FactChecker(vectorstore)
fact_checker.invoke(f"{res.query}: {res.response}")

FactCheckerResponse(score='10', reason='提供された情報は正確であり、電通グループの創業年が1901年であることが正しく記述されています。', reference=['広告業界の成り立ちと電通の変遷'])